## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
file_path = os.path.join("..","Weather_Database","WeatherPy_database.csv")
city_data_df = pd.read_csv(file_path)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Coolum Beach,AU,-26.5333,153.1000,69.12,92,70,7.90,broken clouds
1,1,Avarua,CK,-21.2078,-159.7750,82.45,78,40,3.44,scattered clouds
2,2,Inhambane,MZ,-23.8650,35.3833,76.03,89,1,5.30,clear sky
3,3,North Bend,US,43.4065,-124.2243,48.85,85,75,10.36,broken clouds
4,4,Hilo,US,19.7297,-155.0900,80.28,89,100,6.91,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Coolum Beach,AU,-26.5333,153.1000,69.12,92,70,7.90,broken clouds
1,1,Avarua,CK,-21.2078,-159.7750,82.45,78,40,3.44,scattered clouds
2,2,Inhambane,MZ,-23.8650,35.3833,76.03,89,1,5.30,clear sky
4,4,Hilo,US,19.7297,-155.0900,80.28,89,100,6.91,light rain
5,5,Intipuca,SV,13.1969,-88.0544,94.91,37,16,9.24,few clouds
6,6,Punta Arenas,CL,-53.1500,-70.9167,50.11,62,40,32.21,scattered clouds
10,10,Busselton,AU,-33.6500,115.3333,55.29,86,0,8.97,clear sky
11,11,Hobart,AU,-42.8794,147.3294,52.27,82,20,5.75,few clouds
12,12,Bilma,NE,18.6853,12.9164,86.54,8,0,8.41,clear sky
14,14,Guerrero Negro,MX,27.9769,-114.0611,65.37,59,20,23.80,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                503
City                   503
Country                496
Lat                    503
Lng                    503
Max Temp               503
Humidity               503
Cloudiness             503
Wind Speed             503
Current Description    503
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Coolum Beach,AU,69.12,broken clouds,-26.5333,153.1000,
1,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,
2,Inhambane,MZ,76.03,clear sky,-23.8650,35.3833,
4,Hilo,US,80.28,light rain,19.7297,-155.0900,
5,Intipuca,SV,94.91,few clouds,13.1969,-88.0544,
6,Punta Arenas,CL,50.11,scattered clouds,-53.1500,-70.9167,
10,Busselton,AU,55.29,clear sky,-33.6500,115.3333,
11,Hobart,AU,52.27,few clouds,-42.8794,147.3294,
12,Bilma,NE,86.54,clear sky,18.6853,12.9164,
14,Guerrero Negro,MX,65.37,few clouds,27.9769,-114.0611,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.nan
        print("Hotel not found... skipping.")     

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Coolum Beach,AU,69.12,broken clouds,-26.5333,153.1000,Coolum Beach Resort
1,Avarua,CK,82.45,scattered clouds,-21.2078,-159.7750,Paradise Inn
2,Inhambane,MZ,76.03,clear sky,-23.8650,35.3833,Oceano
4,Hilo,US,80.28,light rain,19.7297,-155.0900,Hilo Hawaiian Hotel
5,Intipuca,SV,94.91,few clouds,13.1969,-88.0544,Tropiclub Playa El Cuco
6,Punta Arenas,CL,50.11,scattered clouds,-53.1500,-70.9167,Hotel Hain
10,Busselton,AU,55.29,clear sky,-33.6500,115.3333,Observatory Guest House
11,Hobart,AU,52.27,few clouds,-42.8794,147.3294,St Ives Hobart Accommodation Tasmania
12,Bilma,NE,86.54,clear sky,18.6853,12.9164,Expeditions Ténére voyages
14,Guerrero Negro,MX,65.37,few clouds,27.9769,-114.0611,"Casa Laguna, Bed & Breakfast"


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))